In [ ]:
import numpy as np
import os
import re
import string
import math
import random
import tensorflow as tf
import keras
import urllib.request
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from keras.utils import np_utils
import time
from wordcloud import WordCloud
from matplotlib import animation
from matplotlib.animation import FuncAnimation

In [ ]:
import urllib.request
!wget https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/miniproject3/data/shakespeare.txt
!wget https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/miniproject3/data/Syllable_dictionary.txt

--2023-03-13 03:21:37--  https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/miniproject3/data/shakespeare.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98029 (96K) [text/plain]
Saving to: ‘shakespeare.txt.2’

shakespeare.txt.2   100%[===================>]  95.73K  --.-KB/s    in 0.002s  

2023-03-13 03:21:37 (44.2 MB/s) - ‘shakespeare.txt.2’ saved [98029/98029]

--2023-03-13 03:21:37--  https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/miniproject3/data/Syllable_dictionary.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request s

## Data Processing

In [ ]:
syllable_dict = {}

with open('Syllable_dictionary.txt') as f:

  for line in f:
    line = line[:-1]
    info = line.split(' ')
    if len(info) != 2:
      if info[1] in ['E1','E2','E3']:
        info.remove(info[1])
      elif info[2] in ['E1','E2','E3']:
        info.remove(info[2])
    syllable_dict[info[0]] = int (info[1])

In [ ]:
big_list = []

with open('shakespeare.txt') as f:
  strng = f.read()

strng = strng.lower()
strng = strng.translate(str.maketrans('', '', string.digits))
strng = strng.translate(str.maketrans('', '', string.punctuation)).replace('\n', ' ').strip()

int_encoding = {}
char_encoding = {}


encoded_strng = []
for i in range(26):
  int_encoding[i] = chr(97+i)
  char_encoding[chr(97+i)] = i

int_encoding[26] = ' '
char_encoding[' '] = 26

for i in range(len(strng)):
  current_char = strng[i]
  encoded_strng.append(char_encoding[current_char])

print(int_encoding)



{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z', 26: ' '}


In [ ]:
# go thru lines
# remove unnecessary stuff
# tokenize with the tweet tokenizer
# loop thru words, make lowercase
# make the dictionary from numbers to characters
#




dataX = []
dataY = []

for i in range(0, len(encoded_strng) - 40-1 , 1):
  seq_in = encoded_strng[i:i + 40]
  seq_out = encoded_strng[i + 40]

  if seq_out==1: #Skip samples where target word is OOV
    continue

  dataX.append(seq_in)
  dataY.append(seq_out)


train_x = np.array(dataX)
train_y = np.array(np_utils.to_categorical(dataY))


In [ ]:
from keras.engine import input_layer

model = keras.Sequential([
    tf.keras.layers.Embedding(28, 27),
    tf.keras.layers.LSTM(100, input_shape=(None, 27)),
    tf.keras.layers.Lambda(lambda x: x / 1),
    tf.keras.layers.Dense(27, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_x, train_y, epochs=4)

Epoch 1/4
2915/2915 [==============================] - 82s 27ms/step - loss: 2.1335 - accuracy: 0.3712
Epoch 2/4
2915/2915 [==============================] - 78s 27ms/step - loss: 1.7843 - accuracy: 0.4591
Epoch 3/4
2915/2915 [==============================] - 79s 27ms/step - loss: 1.6641 - accuracy: 0.4934
Epoch 4/4
2915/2915 [==============================] - 80s 28ms/step - loss: 1.5893 - accuracy: 0.5121


In [ ]:
strng = 'shall i compare thee to a summers day'

converted_strng = []
for char in strng:
  converted_strng.append(char_encoding[char])
num_str = converted_strng

print(num_str)
result_mat = []
for i in range(80):
  token_list = tf.keras.preprocessing.sequence.pad_sequences([num_str], maxlen=81-1, padding='pre')
  predict_x=model.predict(token_list)
  classes_x = np.random.choice(len(predict_x.flatten()), p=predict_x.flatten())
  result_mat.append(classes_x)
  num_str.append(classes_x)


print(result_mat)
print(num_str)

[18, 7, 0, 11, 11, 26, 8, 26, 2, 14, 12, 15, 0, 17, 4, 26, 19, 7, 4, 4, 26, 19, 14, 26, 0, 26, 18, 20, 12, 12, 4, 17, 18, 26, 3, 0, 24]
1/1 [==============================] - 0s 34ms/step
[26, 3, 4, 17, 2, 4, 26, 19, 7, 4, 13, 26, 11, 8, 6, 7, 19, 26, 19, 0, 19, 8, 13, 6, 26, 3, 14, 19, 7, 26, 0, 11, 11, 26, 5, 0, 17, 8, 5, 4, 3, 26, 12, 24, 26, 2, 4, 4, 13, 26, 22, 8, 19, 7, 14, 17, 26, 0, 18, 26, 10, 4, 18, 4, 2, 20, 11, 4, 18, 26, 19, 4, 11, 11, 26, 8, 19, 26, 18, 20]
[18, 7, 0, 11, 11, 26, 8, 26, 2, 14, 12, 15, 0, 17, 4, 26, 19, 7, 4, 4, 26, 19, 14, 26, 0, 26, 18, 20, 12, 12, 4, 17, 18, 26, 3, 0, 24, 26, 3, 4, 17, 2, 4, 26, 19, 7, 4, 13, 26, 11, 8, 6, 7, 19, 26, 19, 0, 19, 8, 13, 6, 26, 3, 14, 19, 7, 26, 0, 11, 11, 26, 5, 0, 17, 8, 5, 4, 3, 26, 12, 24, 26, 2, 4, 4, 13, 26, 22, 8, 19, 7, 14, 17, 26, 0, 18, 26, 10, 4, 18, 4, 2, 20, 11, 4, 18, 26, 19, 4, 11, 11, 26, 8, 19, 26, 18, 20]


In [ ]:
finished_str = []
for i in range(len(np.array(num_str).flatten())):
  finished_str.append(int_encoding[num_str[i]])

print(''.join(finished_str))

shall i compare thee to a summers day derce then light tating doth all farifed my ceen withor as kesecules tell it su


In [ ]:
from keras.engine import input_layer

model = keras.Sequential([
    tf.keras.layers.Embedding(28, 27),
    tf.keras.layers.LSTM(100, input_shape=(None, 27)),
    tf.keras.layers.Lambda(lambda x: x *.75),
    tf.keras.layers.Dense(27, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_x, train_y, epochs=4)

Epoch 1/4
2915/2915 [==============================] - 82s 27ms/step - loss: 2.2190 - accuracy: 0.3501
Epoch 2/4
2915/2915 [==============================] - 79s 27ms/step - loss: 1.9097 - accuracy: 0.4255
Epoch 3/4
2915/2915 [==============================] - 80s 27ms/step - loss: 1.8019 - accuracy: 0.4567
Epoch 4/4
2915/2915 [==============================] - 81s 28ms/step - loss: 1.7334 - accuracy: 0.4732


In [ ]:
strng = 'shall i compare thee to a summers day'

converted_strng = []
for char in strng:
  converted_strng.append(char_encoding[char])
num_str = converted_strng

print(num_str)
result_mat = []
for i in range(80):
  token_list = tf.keras.preprocessing.sequence.pad_sequences([num_str], maxlen=81-1, padding='pre')
  predict_x=model.predict(token_list)
  classes_x = np.random.choice(len(predict_x.flatten()), p=predict_x.flatten())
  result_mat.append(classes_x)
  num_str.append(classes_x)



[18, 7, 0, 11, 11, 26, 8, 26, 2, 14, 12, 15, 0, 17, 4, 26, 19, 7, 4, 4, 26, 19, 14, 26, 0, 26, 18, 20, 12, 12, 4, 17, 18, 26, 3, 0, 24]
1/1 [==============================] - 0s 24ms/step


In [ ]:
finished_str = []
for i in range(len(np.array(num_str).flatten())):
  finished_str.append(int_encoding[num_str[i]])

print(''.join(finished_str))

shall i compare thee to a summers day yoy me priwy dims nut telce out womerines doth of love the sing to held pcuedes


In [ ]:
from keras.engine import input_layer

model = keras.Sequential([
    tf.keras.layers.Embedding(28, 27),
    tf.keras.layers.LSTM(100, input_shape=(None, 27)),
    tf.keras.layers.Lambda(lambda x: x *.25),
    tf.keras.layers.Dense(27, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_x, train_y, epochs=4)

Epoch 1/4
2915/2915 [==============================] - 84s 28ms/step - loss: 2.3435 - accuracy: 0.3220
Epoch 2/4
2915/2915 [==============================] - 80s 27ms/step - loss: 2.0208 - accuracy: 0.3972
Epoch 3/4
2915/2915 [==============================] - 81s 28ms/step - loss: 1.9154 - accuracy: 0.4233
Epoch 4/4
2915/2915 [==============================] - 81s 28ms/step - loss: 1.8468 - accuracy: 0.4437


In [ ]:
strng = 'shall i compare thee to a summers day'

converted_strng = []
for char in strng:
  converted_strng.append(char_encoding[char])
num_str = converted_strng

print(num_str)
result_mat = []
for i in range(80):
  token_list = tf.keras.preprocessing.sequence.pad_sequences([num_str], maxlen=81-1, padding='pre')
  predict_x=model.predict(token_list)
  classes_x = np.random.choice(len(predict_x.flatten()), p=predict_x.flatten())
  result_mat.append(classes_x)
  num_str.append(classes_x)



[18, 7, 0, 11, 11, 26, 8, 26, 2, 14, 12, 15, 0, 17, 4, 26, 19, 7, 4, 4, 26, 19, 14, 26, 0, 26, 18, 20, 12, 12, 4, 17, 18, 26, 3, 0, 24]
1/1 [==============================] - 0s 21ms/step


In [ ]:
finished_str = []
for i in range(len(np.array(num_str).flatten())):
  finished_str.append(int_encoding[num_str[i]])

print(''.join(finished_str))

shall i compare thee to a summers day the thertt iluve tpars evan tht the worchentt waen thify puromt fire   as when 


In [ ]:
from keras.engine import input_layer

model = keras.Sequential([
    tf.keras.layers.Embedding(28, 27),
    tf.keras.layers.LSTM(100, input_shape=(None, 27)),
    tf.keras.layers.Lambda(lambda x: x *1.5),
    tf.keras.layers.Dense(27, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_x, train_y, epochs=4)

Epoch 1/4
2915/2915 [==============================] - 82s 27ms/step - loss: 2.0923 - accuracy: 0.3809
Epoch 2/4
2915/2915 [==============================] - 79s 27ms/step - loss: 1.7408 - accuracy: 0.4729
Epoch 3/4
2915/2915 [==============================] - 81s 28ms/step - loss: 1.6192 - accuracy: 0.5052
Epoch 4/4
2915/2915 [==============================] - 89s 31ms/step - loss: 1.5431 - accuracy: 0.5220


In [ ]:
strng = 'shall i compare thee to a summers day'

converted_strng = []
for char in strng:
  converted_strng.append(char_encoding[char])
num_str = converted_strng

print(num_str)
result_mat = []
for i in range(80):
  token_list = tf.keras.preprocessing.sequence.pad_sequences([num_str], maxlen=81-1, padding='pre')
  predict_x=model.predict(token_list)
  classes_x = np.random.choice(len(predict_x.flatten()), p=predict_x.flatten())
  result_mat.append(classes_x)
  num_str.append(classes_x)



[18, 7, 0, 11, 11, 26, 8, 26, 2, 14, 12, 15, 0, 17, 4, 26, 19, 7, 4, 4, 26, 19, 14, 26, 0, 26, 18, 20, 12, 12, 4, 17, 18, 26, 3, 0, 24]
1/1 [==============================] - 0s 21ms/step


In [ ]:
finished_str = []
for i in range(len(np.array(num_str).flatten())):
  finished_str.append(int_encoding[num_str[i]])

print(''.join(finished_str))


print(len(finished_str))
print(len(strng))

shall i compare thee to a summers day seems in that those thoush i carth detoy the wos with likes you   every with li
117
37
